# Import Data

In [1]:
import sys

## Get the path dinamically to the src folder and add it to the path
sys.path.append("../src")

import importer

train_values, train_labels, test_values = importer.import_data(directory="../Data")

print(train_values.shape)
print(train_labels.shape)
print(test_values.shape)


(86868, 39)
(260601, 2)
(86868, 39)


# Impute Data

In [2]:
import imputator

train_values, train_labels, test_values = imputator.impute_data(train_values, train_labels, test_values)

# Encode Data

In [3]:
import encoder

train_values, train_labels, test_values = encoder.encode(train_values, train_labels, test_values)


## Geo encoder

# Train data Geo Values
geo1 = encoder.dummy_encode(train_values["geo_level_1_id"])
geo2 = encoder.dummy_encode(train_values["geo_level_2_id"])
geo3 = encoder.dummy_encode(train_values["geo_level_3_id"])

# Create a model

In [ ]:
import model

model = model.neural_network(geo1.shape[1], geo2.shape[1], geo3.shape[1])

model.fit(geo3, [geo2, geo1], epochs=10, batch_size=32, validation_split=0.2)
model.save("model.h5")

# Evaluate the model

In [ ]:
import evaluator

evaluator.print_model_summary(model)

# Submit

In [ ]:
import submiter

submiter.submit(model, test_values)
